In [15]:
from langdetect import detect
import pandas as pd
import openpyxl

In [16]:
df = pd.read_csv('/Users/matteo/Work/IndexData_try_NLP/100k_dataset.csv')

In [20]:
df = df[:50000]
df['lang'] = df.apply(lambda x: detect(x['description']), axis=1)

/Users/matteo/Work/IndexData_try_NLP/virtual_env_spacy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:

df.loc[58490].description

'4\x10'

In [21]:
df2 = df[df['lang']== 'en']
#df = df.sample(1000)
df2.shape

(5131, 4)

In [5]:
df['description'] = df['title'] + " " + df['description'] 
#df = df.sample(100)

In [6]:
#///////////////////////////////////////////// SPACY /////////////////////////////////////////////

In [7]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

In [8]:
#!python3 -m spacy download en_core_web_sm

In [9]:
nlp = en_core_web_sm.load()

In [10]:
df['nlp_desc'] = df.apply(lambda x: nlp(x['description']), axis= 1)

In [11]:
def extract_ents(ents):
    res = {}
    for ent in ents:
        res[ent.text] = ent.label_
        
    return res

In [12]:
df['spacy'] = df.apply(lambda x: extract_ents(x['nlp_desc'].ents), axis= 1)

In [13]:
df.drop('nlp_desc', inplace=True, axis= 1)

In [14]:
#/////////////////////////////////////////// Stanford Core NLP /////////////////////////////////////////////

In [15]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import nltk 
#nltk.download('punkt')
from tqdm import tqdm_notebook as tqdm

st = StanfordNERTagger('/Users/matteo/Work/IndexData_try_NLP/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/Users/matteo/Work/IndexData_try_NLP/stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

In [16]:
def filter_class_text(classified_text):

    filtered_classified_text = {}
    for elem in classified_text:
        if elem[1] != 'O':
            filtered_classified_text[elem[0]] = elem[1]
            
    return filtered_classified_text

In [17]:
df['Stanf_tokenized'] = df.apply(lambda x: st.tag(word_tokenize(x['description'])), axis=1)

df['Stanford'] = df.apply(lambda x: filter_class_text(x['Stanf_tokenized']), axis=1)

In [18]:
df.drop('Stanf_tokenized', inplace=True, axis=1)


In [19]:
df

,_id,title,description,lang,spacy,Stanford
19769,350738164,Events/Communications Officer 3985377 | career...,Events/Communications Officer 3985377 | career...,en,"{'Latrobe Valley': 'PERSON', 'Australia': 'GPE...","{'Latrobe': 'LOCATION', 'Valley': 'LOCATION', ..."
4132,346206352,Head Product & Actuarial Solutions,Head Product & Actuarial Solutions head produc...,en,"{'b2b': 'GPE', 'swiss': 'NORP', 'zurich': 'GPE...","{'Head': 'ORGANIZATION', 'Product': 'ORGANIZAT..."
19772,350748856,KS2 TA,KS2 TA School and Role Details A mainstream Pr...,en,"{'KS2 TA School': 'ORG', 'Didsbury': 'GPE', 'S...","{'TA': 'ORGANIZATION', 'School': 'ORGANIZATION..."
19762,350714650,Apprentice Teaching Assistant,Apprentice Teaching Assistant Job Description ...,en,"{'RNN Training': 'ORG', 'Apprenticeship': 'ORG...","{'DBS': 'ORGANIZATION', 'Dearne': 'ORGANIZATIO..."
13862,347370710,Security Engineer,Security Engineer share by email huxley associ...,en,"{'belgium': 'GPE', '6 months': 'DATE', '14': '...",{}
...,...,...,...,...,...,...
14876,349114818,Researcher on Interactions between Fisheries a...,Researcher on Interactions between Fisheries a...,en,"{'15': 'CARDINAL', 'august 2019': 'DATE', '35'...",{}
3111,344930177,Product Security Strategy & Governance Senior ...,Product Security Strategy & Governance Senior ...,en,"{'new jersey': 'GPE', 'cincinnati': 'GPE', 'oh...","{'united': 'LOCATION', 'states': 'LOCATION'}"
18582,343231015,Guest Relations Officer,Guest Relations Officer A fast-growing hospita...,en,{'Larnaca': 'GPE'},{'Larnaca': 'LOCATION'}
15673,343972203,Early morning cleaning operative Lincoln 4 LN1...,Early morning cleaning operative Lincoln 4 LN1...,en,"{'Early morning': 'TIME', 'Lincoln': 'ORG', 'L...","{'Lincoln': 'LOCATION', 'UK': 'LOCATION'}"


In [20]:
#///////////////////////////////////////////// Flair ///////////////////////////////////////////////

In [21]:
import flair
from flair.models import SequenceTagger
from flair.data import Sentence

In [22]:
model = SequenceTagger.load('ner-ontonotes-fast') #.load('ner')
#from flair.data import Sentence

2020-01-01 20:57:07,483 loading file /Users/matteo/.flair/models/en-ner-ontonotes-fast-v0.4.pt


In [23]:
def get_flair_tags(jp):
    s = Sentence(jp)
    model.predict(s)
    entities = s.to_dict(tag_type='ner')['entities']
    tags = {}
    for elem in entities:
        tags[elem['text']] = elem['type']
    
    return tags
    
    

In [24]:
df['flair'] = df.apply(lambda x: get_flair_tags(x['description']), axis=1)

In [25]:
df

,_id,title,description,lang,spacy,Stanford,flair
19769,350738164,Events/Communications Officer 3985377 | career...,Events/Communications Officer 3985377 | career...,en,"{'Latrobe Valley': 'PERSON', 'Australia': 'GPE...","{'Latrobe': 'LOCATION', 'Valley': 'LOCATION', ...","{'Latrobe Valley': 'LOC', 'Central Gippsland.'..."
4132,346206352,Head Product & Actuarial Solutions,Head Product & Actuarial Solutions head produc...,en,"{'b2b': 'GPE', 'swiss': 'NORP', 'zurich': 'GPE...","{'Head': 'ORGANIZATION', 'Product': 'ORGANIZAT...","{'Head Product & Actuarial Solutions': 'ORG', ..."
19772,350748856,KS2 TA,KS2 TA School and Role Details A mainstream Pr...,en,"{'KS2 TA School': 'ORG', 'Didsbury': 'GPE', 'S...","{'TA': 'ORGANIZATION', 'School': 'ORGANIZATION...","{'KS2 TA School': 'ORG', 'Primary School': 'OR..."
19762,350714650,Apprentice Teaching Assistant,Apprentice Teaching Assistant Job Description ...,en,"{'RNN Training': 'ORG', 'Apprenticeship': 'ORG...","{'DBS': 'ORGANIZATION', 'Dearne': 'ORGANIZATIO...","{'HLTA': 'ORG', 'example, National Curriculum,..."
13862,347370710,Security Engineer,Security Engineer share by email huxley associ...,en,"{'belgium': 'GPE', '6 months': 'DATE', '14': '...",{},"{'6 months': 'DATE', '14': 'CARDINAL', '39 hou..."
...,...,...,...,...,...,...,...
14876,349114818,Researcher on Interactions between Fisheries a...,Researcher on Interactions between Fisheries a...,en,"{'15': 'CARDINAL', 'august 2019': 'DATE', '35'...",{},"{'35': 'CARDINAL', '2 years': 'DATE', 'english..."
3111,344930177,Product Security Strategy & Governance Senior ...,Product Security Strategy & Governance Senior ...,en,"{'new jersey': 'GPE', 'cincinnati': 'GPE', 'oh...","{'united': 'LOCATION', 'states': 'LOCATION'}","{'california': 'GPE', 'johnson & johnson.': 'O..."
18582,343231015,Guest Relations Officer,Guest Relations Officer A fast-growing hospita...,en,{'Larnaca': 'GPE'},{'Larnaca': 'LOCATION'},{'Larnaca': 'GPE'}
15673,343972203,Early morning cleaning operative Lincoln 4 LN1...,Early morning cleaning operative Lincoln 4 LN1...,en,"{'Early morning': 'TIME', 'Lincoln': 'ORG', 'L...","{'Lincoln': 'LOCATION', 'UK': 'LOCATION'}","{'Lincoln 4 LN1': 'ORG', 'Lincoln city': 'GPE'..."


In [48]:
#df.drop('lang', inplace=True, axis=1)
#df.drop('title', inplace=True, axis=1)

df.to_excel('/Users/matteo/Work/IndexData_try_NLP/ner_res.xlsx', index= False, sheet_name='results')

In [26]:
def get_org_ents(my_dict):
    my_set = set()
    for (key, value) in my_dict.items():
        if (value == 'ORG') | (value == 'ORGANIZATION') :
            my_set.add(key)
            
    return my_set


def get_loc_ents(my_dict):
    my_set = set()
    for (key, value) in my_dict.items():
        if (value == 'GPE') | (value == 'LOCATION'):
            my_set.add(key)
            
    return my_set


def get_date_ents(my_dict):
    my_set = set()
    for (key, value) in my_dict.items():
        if (value == 'DATE'):
            my_set.add(key)
            
    return my_set


def get_mon_ents(my_dict):
    my_set = set()
    for (key, value) in my_dict.items():
        if (value == 'MONEY'):
            my_set.add(key)
            
    return my_set

def get_other_ents(my_dict):
    my_list = []
    for (key, value) in my_dict.items():
        if value not in ('MONEY','ORG','ORGANIZATION', 'DATE', 'GPE', 'LOCATION'):
            dicted = {}
            dicted[key] = value
            my_list.append(dicted)
            
    return my_list

In [28]:
df['spacy_ORG'] = df.apply(lambda x: get_org_ents(x['spacy']), axis=1)
df['Stanford_ORG'] = df.apply(lambda x: get_org_ents(x['Stanford']), axis=1)
df['flair_ORG'] = df.apply(lambda x: get_org_ents(x['flair']), axis=1)

df['spacy_LOC'] = df.apply(lambda x: get_loc_ents(x['spacy']), axis=1)
df['Stanford_LOC'] = df.apply(lambda x: get_loc_ents(x['Stanford']), axis=1)
df['flair_LOC'] = df.apply(lambda x: get_loc_ents(x['flair']), axis=1)

df['spacy_DATE'] = df.apply(lambda x: get_date_ents(x['spacy']), axis=1)
df['Stanford_DATE'] = df.apply(lambda x: get_date_ents(x['Stanford']), axis=1)
df['flair_DATE'] = df.apply(lambda x: get_date_ents(x['flair']), axis=1)

df['spacy_MON'] = df.apply(lambda x: get_mon_ents(x['spacy']), axis=1)
df['Stanford_MON'] = df.apply(lambda x: get_mon_ents(x['Stanford']), axis=1)
df['flair_MON'] = df.apply(lambda x: get_mon_ents(x['flair']), axis=1)

df['spacy_others'] = df.apply(lambda x: get_other_ents(x['spacy']), axis=1)
df['Stanford_others'] = df.apply(lambda x: get_other_ents(x['Stanford']), axis=1)
df['flair_others'] = df.apply(lambda x: get_other_ents(x['flair']), axis=1)

In [29]:
df.drop('spacy', inplace=True, axis=1)
df.drop('Stanford', inplace=True, axis=1)
df.drop('flair', inplace=True, axis=1)

df.to_excel('/Users/matteo/Work/IndexData_try_NLP/ner_res.xlsx', index= False, sheet_name='results')

In [102]:
for elem in list(df.Stanford):
    ms = set()
    for (k,v) in elem.items():
        ms.add(v)
        
ms



{'LOCATION', 'ORGANIZATION', 'PERSON'}

In [68]:
org_set = set()
for (key, value) in a.items():
    if (value == 'ORG') | (value == 'ORGANIZATION') :
        org_set.add(key)

In [69]:
org_set

{'Electronics Engineer',
 'adresse',
 'ein',
 'f/m/d',
 'ihrer personenbezogenen',
 'mit',
 'uns gerne',
 'unseren aktuellen anzeigen'}